Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

**IMPORTANT: DO NOT COPY OR SPLIT CELLS.** If you do, you'll mess the autograder. If need more cells to work or test things out, create a new cell. You may add as many new cells as you need.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and group below:

In [1]:
COURSE = "Unsupervised Learning 2021"
GROUP = "D8A"
NAME = "Tokiyomi" # Match your GitHub Classroom ID

---

# **Warning**:

Make sure your whole notebooks executes in a reasonable amount of time (< 10 min), less it will not be graded.

In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba
from collections import Counter
import matplotlib.pyplot as plt
from itertools import product
import time

# Exercise 1 (2 pt)

Compute the simple matching coefficient, cosine similarity, and the Jaccard coefficient, between the two sets {A,B,C} and {A,C,D,E}.

To do so, modify the functions for each similarity to work with sets instead of vectors.

Compare your functions output with a manual calculation.

In [3]:
def smc(A, B):
    # Intersection 
    return len(A&B) 
    
def cosine_s(A, B):
    # Intersection / sqrt(|A| x |B|)
    return len(A&B) / np.sqrt(len(A)*len(B))

def jaccard(A, B):
    # Intersection / Union
    return len(A&B)/len(A|B)

In [4]:
s1 = {'A', 'B', 'C'}
s2 = {'A', 'C', 'D', 'E'}

print(f'Simple matching coefficient: {smc(s1, s2)}')
print(f'Cosine similarity: {cosine_s(s1, s2)}')
print(f'Jaccard index: {jaccard(s1, s2)}')

Simple matching coefficient: 2
Cosine similarity: 0.5773502691896258
Jaccard index: 0.4


# Exercise 2 (3 pt)

## The data set

Extraction was done by Barry Becker from the 1994 Census database. A set of reasonably clean records was extracted using the following conditions: ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))

Prediction task is to determine whether a person makes over 50K a year.


Listing of attributes:

- class: >50K, <=50K.

- age: continuous.
- workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- fnlwgt: continuous.
- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- education-num: continuous.
- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- sex: Female, Male.
- capital-gain: continuous.
- capital-loss: continuous.
- hours-per-week: continuous.
- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [5]:
# Load the complete data set
with open('adult.names', 'r') as f:
    lines = [l.strip() for l in f.readlines()][-14:]
cols = [l.split(':')[0] for l in lines] + ['y']
cols
df = pd.read_csv('adult.data', names=cols, na_values='?', skipinitialspace=True)
df = df.dropna()
# added reset index
df.reset_index(drop=True, inplace=True)
print(df.shape)
df.head()

(30162, 15)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,y
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## Part 1 (2 pt)

Using the _Adults Data Set_ from the _UCI Machine Learning Repository_ (provided), create a data set containing only the categorical attributes. Compute the nearest neighbor for each data point using 
- (a) the SMC (1 pt)
- (b) inverse ocurrence frequency measure (1 pt)

Compute the number of cases where there is a match on the class label, store them into `match_smc` and `match_iof`.
When there are ties among NN, the 1st NN match is undefined and depends on the ordering of the data, but the distributions of distances is well defined. Use Counter class to find the distribution of distances and store the dictionaries in `dist_smc` and `dist_iof`.

Hints: 
- Do not try to compute the full distance matrix, you may run into memory issues.
- The data set is large, even at 10%, so pure Python loops will be slow, try using numba for just in time compilation. Sklearn with cythonized custom metrics may work, but I've had issues since sklearn tends to report a point as its self NN, not necessarly the first one, if many neighbors with the same distance exist.
- Test your code with a small sample of the data to avoid waiting much time for completion during testing.
- Note: This hints were valid for the kdd cup data set, consisiting of ~5million records, memory issues may no longer apply to the significantly smaller bank dataset

Extra points if able to find a way to perform the excercise for the full KDD Cup data set in a reasonable time, using 100k rows, for SMC, my personal laptop takes ~ 5 min.

In [6]:
y = pd.get_dummies(df['y'])['<=50K']
y = y.to_numpy()

### Part a

In [7]:
# Categorical dataset
cat_df = df[['workclass','education','marital-status','occupation','relationship','race','sex','native-country']]

# usar sample
sample = cat_df
sample = sample.apply(lambda x: pd.factorize(x)[0])
sample = sample.to_numpy()


# SMC
# compute NN
@jit(nopython=True)
def compute_smc(sample, target):
    smc_distances = np.zeros((len(sample)))
    
    matches_count = 0

    for i in range(len(sample)):
        smc_dist = -1
        smc_idx = -1
        for j in range(len(sample)):
            if i == j:
                continue
            dist = ((sample[i] == sample[j]).sum()) / (len(sample[i]))
            if dist > smc_dist:
                smc_dist = dist
                smc_idx = j
        if target[i] == target[smc_idx]:
            matches_count += 1

        smc_distances[i] = smc_dist
        

    return (1 - smc_distances, matches_count)

In [8]:
# COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME
start = time.time()
dist_smc, match_smc = compute_smc(sample, y)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))
dist_smc = Counter(dist_smc) 

Elapsed (with compilation) = 153.80569458007812


In [9]:
print(match_smc)
print(dist_smc)

23244
Counter({0.0: 25005, 0.125: 4612, 0.25: 521, 0.375: 24})


In [10]:
print(f'Matches using SMC or equivalent: {match_smc}, expected arround: 20k-25k')
print(f'The distribution of distances is: {dist_smc}')
print('Expected:\n{0.0: 25005,\n 0.125: 4612,\n 0.25: 521,\n 0.375: 24}')

Matches using SMC or equivalent: 23244, expected arround: 20k-25k
The distribution of distances is: Counter({0.0: 25005, 0.125: 4612, 0.25: 521, 0.375: 24})
Expected:
{0.0: 25005,
 0.125: 4612,
 0.25: 521,
 0.375: 24}


### Part b

In [11]:
result = []
for feature in range(len(sample.T)):
    result_feature = []
    unique_values = np.unique(sample.T[feature])
    for unique in unique_values:
        inv_freq =  (len(sample.T[0]) / (sample.T[feature]==unique).sum())**2
        result_feature.append(inv_freq)
    result.append(result_feature)

In [12]:
indices_true, = np.where(sample[0] == sample[1])
indices_true

array([1, 5, 6, 7], dtype=int64)

In [17]:
def compute_dist(dic,i,j):
    indices_true, = np.where(sample[i] == sample[j])
    suma = 0
    for idx in indices_true:
        inv_freq = dic[idx][sample[i][idx]]
        suma += inv_freq
    return suma

In [18]:
compute_dist(result,0,1)

40.50344794987936

In [19]:
@jit(nopython=True)
def compute_iof(sample, target):
    # create dict of inv frequencies
    result = []
    for feature in range(len(sample.T)):
        result_feature = []
        unique_values = np.unique(sample.T[feature])
        for unique in unique_values:
            inv_freq =  (len(sample.T[0]) / (sample.T[feature]==unique).sum())**2
            result_feature.append(inv_freq)
        result.append(result_feature)
    # compute NN
    iof_distances = np.zeros((len(sample)))
    matches_count = 0
    for i in range(len(sample)):
        iof_dist = -1
        iof_idx = -1
        for j in range(len(sample)):
            if i == j:
                continue
            # compute inv freq
            indices_true, = np.where(sample[i] == sample[j])
            suma = 0
            for idx in indices_true:
                inv_freq = result[idx][sample[i][idx]]
                suma += inv_freq
            dist = suma
            if dist > iof_dist:
                iof_dist = dist
                iof_idx = j
                
        if target[i] == target[iof_idx]:
            matches_count += 1

        iof_distances[i] = iof_dist
        
    return (iof_distances, matches_count)


In [20]:
# COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME
start = time.time()
dist_iof, match_iof = compute_iof(sample, y)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

Elapsed (with compilation) = 342.3478891849518


In [21]:
dist_iof = Counter(dist_iof)

In [22]:
print(f'Matches using IOF or equivalent: {match_iof}, expected arround: 20k-25k')
print(f'The top 5 distances are: {dist_iof.most_common(5)}')
print('Expected:\n(82.44464251281099, 803),\n (109.87817458743599, 448),\n (261.80004755602187, 368),\n (93.44879430104449, 345),\n (394.57073723935673, 332)')

Matches using IOF or equivalent: 23219, expected arround: 20k-25k
The top 5 distances are: [(82.44464251281097, 803), (109.878174587436, 448), (261.80004755602187, 368), (93.44879430104447, 345), (394.5707372393568, 332)]
Expected:
(82.44464251281099, 803),
 (109.87817458743599, 448),
 (261.80004755602187, 368),
 (93.44879430104449, 345),
 (394.57073723935673, 332)


## Part 2 (1 pt)

Now, create a data set using only the quantitative attributes of the data set. Use the $L_p$ norm for values $p=1,2,\infty$ to find the nearest neighbors and count the matches. Store the matches into `match_l1`, `match_l2`, and `match_linf` respectively and distributions into `dist_l1`, `dist_l2`, and `dist_linf`. Use sklearn for this part of the assignment.

In [23]:
# YOUR CODE HERE
num_df = df[['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']]
num_df = num_df.to_numpy()

In [24]:
def get_matches_and_dist(df, target, p):
    nbrs = NearestNeighbors(n_neighbors=2, p=p).fit(df)
    distances, indices = nbrs.kneighbors(df)
    distribution = Counter(distances[:,1])
    matches_count = (target == target[indices[:,1]]).sum()
    return matches_count, distribution

In [25]:
match_l1, dist_l1 = get_matches_and_dist(num_df,y, p=1)
match_l2, dist_l2 = get_matches_and_dist(num_df,y, p=2)
match_linf, dist_linf = get_matches_and_dist(num_df,y, p=np.inf)

In [26]:
print(f'Matches are: Manhattan: {match_l1}, Euclidean: {match_l2}, Chebyshev: {match_linf}')
print('Expected (arround): Manhattan: 22097, Euclidean: 21984, Chebyshev: 21826')
print(f'Distributions most commont:\n Manhattan: {dist_l1.most_common(1)},\n Euclidean: {dist_l2.most_common(1)},\n Chebyshev: {dist_linf.most_common(1)}')
print('Expected:\n Manhattan: [(5.0, 884)],\n Euclidean: [(1.0, 837)],\n Chebyshev: [(10.0, 2507)]')

Matches are: Manhattan: 22091, Euclidean: 21969, Chebyshev: 21854
Expected (arround): Manhattan: 22097, Euclidean: 21984, Chebyshev: 21826
Distributions most commont:
 Manhattan: [(5.0, 884)],
 Euclidean: [(1.0, 837)],
 Chebyshev: [(10.0, 2507)]
Expected:
 Manhattan: [(5.0, 884)],
 Euclidean: [(1.0, 837)],
 Chebyshev: [(10.0, 2507)]


## Part 3 (2 pts)

Repeat for the complete data set. Implement and use the mixed-attribute function, un-normalized, and transform the numerical distances to a similaroty using $s = 1/(1+d)$. Use euclidean distance for numerical attributes. Save matches and distribution into `match_mix` and `dist_mix`.



Tip: Continue the numba approach to build a custom similarity metric.

In [27]:
@jit(nopython=True)
def compute_mixed(sample_c, sample_r, target):
    sim_measures = np.zeros((len(sample_c)))
    #sim_indices = np.zeros((len(sample_c)))
    matches_count = 0
    # set lambda ad the fraction of real features, should be 6/14
    l = len(sample_r[0])/(len(sample_c[0])+len(sample_r[0]))

    for i in range(len(sample_c)):
        max_sim = -1
        sim_idx = -1
        for j in range(len(sample_c)):
            if i == j:
                continue
            sim_c = (((sample_c[i] == sample_c[j]).sum()) / (len(sample_c[i])))

            dist_r = 0
            for z in range(len(sample_r[0])):
                dist_r += (sample_r[i][z]-sample_r[j][z])**2
            dist_r = np.sqrt(dist_r)
            sim_r = 1/(1+dist_r)

            sim_rc = l*sim_r + (1-l)*sim_c
            if sim_rc > max_sim:
                max_sim = sim_rc
                sim_idx = j
            
        if target[i]==target[sim_idx]:
            matches_count += 1

        sim_measures[i] = max_sim
        #sim_indices[i] = sim_idx

    return (sim_measures, matches_count)

In [28]:
# COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME
start = time.time()
a,b=compute_mixed(sample[:], num_df[:], y[:])
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

Elapsed (with compilation) = 161.8349461555481


In [29]:
dist_mix = Counter(a)
dist_mix.most_common(4)

[(0.6428571428571428, 314),
 (0.7142857142857143, 170),
 (0.9285714285714286, 131),
 (0.6060915267313265, 104)]

In [30]:
match_mix = b
match_mix

23311

In [31]:
print(f'Matches: {match_mix}')
print('Expected (arround): 22520')
print(f'Distribution 4 most common:\n {dist_mix.most_common(4)}')
print('Expected:\n [(0.6904761904761905, 837),\n (0.5133882356845012, 439),\n (0.6190476190476191, 426),\n (1.0, 404)]')

Matches: 23311
Expected (arround): 22520
Distribution 4 most common:
 [(0.6428571428571428, 314), (0.7142857142857143, 170), (0.9285714285714286, 131), (0.6060915267313265, 104)]
Expected:
 [(0.6904761904761905, 837),
 (0.5133882356845012, 439),
 (0.6190476190476191, 426),
 (1.0, 404)]


# Exercise 3 (4 pts)

## Part 1 (1 pt)

Implement the edit or Levenshtein distance. Provided start code is a sugestion, you may implement from scratch.

In [7]:
def leveshtein_d(src, dest):
    n = len(src)
    m = len(dest)

    if type(src) is str:
        src = list(src)
    if type(dest) is str:
        dest = list(dest)
        
    
    # for all i and j, d[i,j] will hold the Levenshtein distance
    # between the first i characters of src
    # and the first j characters of dest
    # note that d has (n+1)*(m+1) values to account for the
    # empty string

    D = np.zeros((n+1,m+1), dtype=int)

    for i in range(1,n+1):
        D[i,0] = i
    for j in range(1,m+1):
        D[0,j] = j

    for i in range(1,n+1):
        for j in range(1,m+1):
            if src[i-1] == dest[j-1]:
                D[i][j] = min(D[i-1][j] + 1, D[i][j-1] + 1, D[i-1][j-1])
            else:
                D[i][j] = min(D[i-1][j] + 1, D[i][j-1] + 1, D[i-1][j-1] +1)
    print(D)

    return D[n,m]

In [8]:
print(f'Leveshtein distance between "sitting" and "kitten" should be 3, calculated distance: {leveshtein_d("sitting","kitten")}')

[[0 1 2 3 4 5 6]
 [1 1 2 3 4 5 6]
 [2 2 1 2 3 4 5]
 [3 3 2 1 2 3 4]
 [4 4 3 2 1 2 3]
 [5 5 4 3 2 2 3]
 [6 6 5 4 3 3 2]
 [7 7 6 5 4 4 3]]
Leveshtein distance between "sitting" and "kitten" should be 3, calculated distance: 3


## Part 2 (1 pt)

Impement the LCSS distance. The function must return the distance and the set of all common subsequences as tuples.

In [10]:
def lccs_d(x, y):
    n = len(x)
    m = len(y)

    if type(x) is str:
        x = list(x)
    if type(y) is str:
        y = list(y)

    # The matrix will hold initial values D[0,j], D[i,0]
    # which mean nothing (distance to the empty vector)
    # so set them to 0
    D = np.zeros((n+1,m+1), dtype=int)

    for i in range(1,n+1):
        D[i,0] = 0

    for j in range(1,m+1):
        D[0,j] = 0

    for i in range(1,n + 1):
        for j in range(1,m + 1):
            if x[i-1] == y[j-1]:
                D[i][j] = D[i-1][j-1]+1
            else:
                D[i][j] = max(D[i-1][j], D[i][j-1])
    print(D)

    # Apply traceback to find set LCCS
    def backtrack(n, m):
        # construct a set to store possible LCS
        s = set()
    
        # If we reaches end of either string, return a empty set
        if n == 0 or m == 0:
            s.add("")
            return s
    
        # If the last characters of X and Y are same
        if x[n - 1] == y[m - 1]:
            # recurssion
            tmp = backtrack(n - 1, m - 1)
    
            # append current character to all possible LCS of substring X[0..n-2] and Y[0..m-2].
            for string in tmp:
                s.add(string + x[n - 1])
    
        # If the last characters of X and Y are not same
        else:
    
            # If LCS can be constructed from top side of the matrix, recurse for X[0..n-2] and Y[0..m-1]
            if D[n - 1][m] >= D[n][m - 1]:
                s = backtrack(n - 1, m)
    
            # If LCS can be constructed from left side of the matrix, recurse for X[0..n-1] and Y[0..m-2]
            if D[n][m - 1] >= D[n - 1][m]:
                tmp = backtrack(n, m - 1)
    
                # merge two sets if D[n-1][m] == L[n][m-1]
                for i in tmp:
                    s.add(i)
        return s

    lcs_set = backtrack(n, m)

    lcs_set = set(map(tuple, lcs_set))

    return D[n, m], lcs_set

In [11]:
lccs_d('GAC','AGCAT')

[[0 0 0 0 0 0]
 [0 0 1 1 1 1]
 [0 1 1 1 2 2]
 [0 1 1 2 2 2]]


(2, {('A', 'C'), ('G', 'A'), ('G', 'C')})

In [12]:
d = lccs_d('GAC','AGCAT')
print(f'The LCSS has length {d[0]}, expected value is 2.')
print(f'The set of LCCSs is {d[1]}')
print("The expected set is: {('G', 'C'), ('A', 'C'), ('G', 'A')}")

[[0 0 0 0 0 0]
 [0 0 1 1 1 1]
 [0 1 1 1 2 2]
 [0 1 1 2 2 2]]
The LCSS has length 2, expected value is 2.
The set of LCCSs is {('A', 'C'), ('G', 'C'), ('G', 'A')}
The expected set is: {('G', 'C'), ('A', 'C'), ('G', 'A')}


## Part 3 (1 pt)

Implement the DTW distance.

In [44]:
def dtw_d(x, y):

    n = len(x)
    m = len(y)

    D = np.zeros((n+1, m+1), dtype=int)

    for i in range(1,n+1):
        D[i,0] = i
    for j in range(1,m+1):
        D[0,j] = j

    #D[0, 0] = 0 # starting position
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = abs(x[i-1] - y[j-1])
            last_min = min(D[i-1][j], D[i][j-1], D[i-1][j-1])
            D[i, j] = cost + last_min
    print(D)
    return D[n, m]


In [45]:
d = dtw_d([1,2,3], [1,2,3,5])
print(f'The DTW distance for the example is {d}, expected value is 2.')

[[0 1 2 3 4]
 [1 0 1 3 7]
 [2 1 0 1 4]
 [3 3 1 0 2]]
The DTW distance for the example is 2, expected value is 2.


Check your implementations by computing by hand each of the lcss and leveshtein distances for the pairs **(1 pt)**, and compare them to the programmed solutions (no need to provide calulations):
- ababcabc, babcbc
- cbacbacba, acbacbacb

In [9]:
print(leveshtein_d('ababcabc', 'babcbc'))
print(leveshtein_d('cbacbacba', 'acbacbacb'))
print(lccs_d('ababcabc', 'babcbc'))
print(lccs_d('cbacbacba', 'acbacbacb'))

[[0 1 2 3 4 5 6]
 [1 1 1 2 3 4 5]
 [2 1 2 1 2 3 4]
 [3 2 1 2 2 3 4]
 [4 3 2 1 2 2 3]
 [5 4 3 2 1 2 2]
 [6 5 4 3 2 2 3]
 [7 6 5 4 3 2 3]
 [8 7 6 5 4 3 2]]
2
[[0 1 2 3 4 5 6 7 8 9]
 [1 1 1 2 3 4 5 6 7 8]
 [2 2 2 1 2 3 4 5 6 7]
 [3 2 3 2 1 2 3 4 5 6]
 [4 3 2 3 2 1 2 3 4 5]
 [5 4 3 2 3 2 1 2 3 4]
 [6 5 4 3 2 3 2 1 2 3]
 [7 6 5 4 3 2 3 2 1 2]
 [8 7 6 5 4 3 2 3 2 1]
 [9 8 7 6 5 4 3 2 3 2]]
2
[[0 0 0 0 0 0 0]
 [0 0 1 1 1 1 1]
 [0 1 1 2 2 2 2]
 [0 1 2 2 2 2 2]
 [0 1 2 3 3 3 3]
 [0 1 2 3 4 4 4]
 [0 1 2 3 4 4 4]
 [0 1 2 3 4 5 5]
 [0 1 2 3 4 5 6]]
(6, {('b', 'a', 'b', 'c', 'b', 'c')})
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 1 1 1 1]
 [0 0 1 2 2 2 2 2 2 2]
 [0 1 1 2 3 3 3 3 3 3]
 [0 1 2 2 3 4 4 4 4 4]
 [0 1 2 3 3 4 5 5 5 5]
 [0 1 2 3 4 4 5 6 6 6]
 [0 1 2 3 4 5 5 6 7 7]
 [0 1 2 3 4 5 6 6 7 8]
 [0 1 2 3 4 5 6 7 7 8]]
(8, {('c', 'b', 'a', 'c', 'b', 'a', 'c', 'b')})


# Exercise 4 (1 pt)

Compute the cosine similarity between the two sentences, store into `cos`:
- The sly fox jumped over the lazy dog.
- The dog jumped at the intruder.

In [40]:
# YOUR CODE HERE
def vectorize(text, word_set):
    vector = []
    for i in word_set:
        count = 0    
        for item in text:
            if i == item:
                count += 1
        vector.append(count)
    return np.array(vector)

def cos_sim(text1,text2):
    # compute the union of the text
    A, B = text1.lower().split(' '), text2.lower().split(' ') 
    word_set = set(A) | set(B)
    print(word_set)
    #vectorize
    a = vectorize(A, word_set)
    print(a)
    b = vectorize(B, word_set)
    print(b)
    # compute numerical cos sim
    return (a@b)/(np.linalg.norm(a)*np.linalg.norm(b))

cos = cos_sim('The sly fox jumped over the lazy dog','The dog jumped at the intruder')
cos

{'lazy', 'intruder', 'the', 'dog', 'at', 'over', 'jumped', 'fox', 'sly'}
[1 0 2 1 0 1 1 1 1]
[0 1 2 1 1 0 1 0 0]


0.6708203932499369

In [41]:
print(f'The cosine similarity is {cos}, expected: {6/np.sqrt(80)}')

The cosine similarity is 0.6708203932499369, expected: 0.6708203932499369


# Exercise 5 (ungraded)

## Part 1

Assume $Edit(\bar{X},\bar{Y})$ represents the cost of transforming string $\bar{X}$ to $\bar{Y}$. Show that $Edit(\bar{X},\bar{Y})$ and $Edit(\bar{Y},\bar{X})$ are the same, as long as the insertion and deletion costs are the same.

Using the leveshtein distance:  leveshtein_d(X,Y)=D and leveshtein_d(Y,X)=D_Transpose 

Therefore, D(len(X),len(Y)) = D_Transpose(len(Y),len(X))

In [42]:
leveshtein_d('ababcabc', 'babcbc')

[[0 1 2 3 4 5 6]
 [1 1 1 2 3 4 5]
 [2 1 2 1 2 3 4]
 [3 2 1 2 2 3 4]
 [4 3 2 1 2 2 3]
 [5 4 3 2 1 2 2]
 [6 5 4 3 2 2 3]
 [7 6 5 4 3 2 3]
 [8 7 6 5 4 3 2]]


2

In [43]:
leveshtein_d('babcbc','ababcabc')

[[0 1 2 3 4 5 6 7 8]
 [1 1 1 2 3 4 5 6 7]
 [2 1 2 1 2 3 4 5 6]
 [3 2 1 2 1 2 3 4 5]
 [4 3 2 2 2 1 2 3 4]
 [5 4 3 3 2 2 2 2 3]
 [6 5 4 4 3 2 3 3 2]]


2

## Part 2

Show that $Edit(\vec{x}_i,\vec{y}_j)$, $LCSS(\vec{x}_i,\vec{y}_j)$, and $DTW(\vec{x}_i,\vec{y}_j)$ are all monotonic functions in $i$ and $j$.

YOUR ANSWER HERE

## Part 3

Suppose that insertion and deletion costs are 1, and replacement costs are 2 units for the edit distance. Show that the optimal edit distance between two strings can be computed only with insertion and deletion operations. Under the aftermentioned cost assumptions, show that the optimal edit distance can be expressed as a function of the optimal LCSS distance and the length of the two strings. 

YOUR ANSWER HERE